## **3.1 ทดลองซ้ำและทำการปรับพารามิเตอร์ให้มากกว่าใน lecture หรือลอง generate data ที่แตกต่างจากที่เรียน** ##

สร้างข้อมูลขึ้นมาเองหรือเรียกใช้ชุดข้อมูลที่มี

In [3]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


# Load datasets
data_10sample = pd.read_csv('../../Data/sin_noisy_10sample.csv')
data_20sample = pd.read_csv('../../Data/sin_noisy_20sample.csv')
data_40sample = pd.read_csv('../../Data/sin_noisy_40sample.csv')
data_80sample = pd.read_csv('../../Data/sin_noisy_80sample.csv')

# Combine datasets into a dictionary
data_dict = {
    10: data_10sample,
    20: data_20sample,
    40: data_40sample,
    80: data_80sample
}



In [4]:
# Custom MSE function
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

#กำหนด degree ที่ต้องการ
degrees = [1,3,8,15,20]

results = []

#วนลูปผ่านแต่ละชุดข้อมูลใน data_dict
for n_samples, data in data_dict.items():
    X = data[['x']].values
    y = data['noisy_y'].values
    
    #วนลูปผ่านแต่ละ polynomial degree ใน degrees
    for degree in degrees:
        poly = PolynomialFeatures(degree)
        X_poly = poly.fit_transform(X)
        
        model = LinearRegression()
        model.fit(X_poly, y)
        
        # Predictions on the training set
        y_pred_train = model.predict(X_poly)
        rmse_train = np.sqrt(mean_squared_error(y, y_pred_train))
        
        # Predictions using cross-validation
        scores = cross_val_score(model, X_poly, y, scoring='neg_mean_squared_error', cv=10)
        rmse_cv = np.sqrt(-scores.mean())
        
        # Record the results
        results.append({
            'n_samples': n_samples,
            'degree': degree,
            'rmse_train': rmse_train,
            'rmse_cv': rmse_cv
        })

results_df = pd.DataFrame(results)

# Display the results
results_df = results_df.round(2)
results_df


,n_samples,degree,rmse_train,rmse_cv
0,10,1,0.57,0.75
1,10,3,0.23,0.43
2,10,8,0.06,19.04
3,10,15,0.00,42.66
4,10,20,0.00,58.57
5,20,1,0.43,0.55
6,20,3,0.27,0.38
7,20,8,0.19,0.84
8,20,15,0.09,3052.76
9,20,20,0.00,75133.48
